In [1]:
!mkdir -p out_demog

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import json
import sparse
import pandas as pd
import numpy as np
import joblib

In [4]:
df_enc = pd.read_csv('data_win/enc.csv').set_index('hosp_id')

In [5]:
df_static = df_enc[['age', 'sex', 'race']].rename(columns={'age': 'age_value', 'sex': 'sex_value', 'race': 'race_value'})

In [6]:
df_static.to_csv('out_demog/hosp_data_raw.csv')

In [7]:
discretization_bins = json.load(open('/data/covid/static/out/discretization.json', 'r'))
feature_names = json.load(open('/data/covid/static/out/s_all.feature_names.json', 'r'))

In [8]:
discretization_bins

{'age_value': [18.0, 47.0, 59.0, 67.0, 76.0, 102.0],
 'sex_value': None,
 'race_value': None}

In [9]:
cols_useful = list(discretization_bins.keys())

In [10]:
df_static = df_static[cols_useful]

In [ ]:
df_static

In [12]:
from steps_2 import *
from helpers import *

In [13]:
df = df_static

In [14]:
df_by_cols = [df[col] for col in df.columns]

In [15]:
out_0 = [smart_qcut_bins((x, discretization_bins[x.name])) for x in tqdm(df_by_cols)]

100%|██████████| 3/3 [00:00<00:00,  3.71it/s]


In [16]:
cols_data, dis_bins = zip(*out_0)

In [17]:
out = [dummify(z) for z in tqdm(cols_data)]

100%|██████████| 3/3 [00:00<00:00, 105.80it/s]


In [18]:
df_features = pd.concat(out, axis=1).sort_index(axis=1)

In [19]:
# Drop any values='missing' for now
df_features = df_features.loc[:, [col for col in df_features.columns if 'missing' not in col]]

In [20]:
# Make sure same features
set(df_features.columns) <= set(feature_names)

True

In [21]:
df_features = df_features.reindex(columns=feature_names, fill_value=0)

In [ ]:
df_features

In [23]:
sdf = df_features.astype(pd.SparseDtype(int, fill_value=0))
X_all = sparse.COO(sdf.sparse.to_coo())

In [24]:
sparse.save_npz('out_demog/X_all.npz', X_all)
df_features[[]].to_csv('out_demog/X_all.IDs.csv')
# json.dump(list(X_all_feature_names), open('out_meds/X_all.feature_names.json', 'w'), sort_keys=True)
# json.dump(discretization_bins, open('out_meds/discretization.json', 'w'))

In [25]:
import sparse, json

In [26]:
s = sparse.load_npz('out_demog/X_all.npz').todense()

In [27]:
s.shape

(10178, 15)

In [28]:
cols = feature_names
df_s = pd.DataFrame(s, columns=cols)
df_s.index = df_static.index

In [29]:
df_s.to_csv('out_demog/static-features.csv')

In [ ]:
df_s